<h1 style="text-align: center;">Computational Theory Tasks</h1>

<div style="display: flex; justify-content: center;">
  <div style="display: flex; flex-direction: column; gap: 12px; max-width: 800px; width: 100%;">

<div style="display: flex; align-items: stretch; gap: 10px;">
      <div style="flex: 0 0 140px; background-color: #5C9E75; color:#FFFFFF; padding:10px; border-radius:8px; font-size:16px; text-align: center;">
        <strong>Test Overview</strong>
      </div>
      <div style="flex: 1; border: 3px solid #5C9E75; padding:12px; border-radius:8px; font-size:13px; color: inherit; background-color: transparent;">
        Explains the tests used to validate each solution, covering the inputs, expected results, and the rationale.
      </div>
</div>

<div style="display: flex; align-items: stretch; gap: 10px;">
      <div style="flex: 0 0 140px; background-color: #4F83CC; color:#FFFFFF; padding:10px; border-radius:8px; font-size:16px; text-align: center;">
        <strong>Sources</strong>
      </div>
      <div style="flex: 1; border: 3px solid #4F83CC; padding:12px; border-radius:8px; font-size:13px; color: inherit; background-color: transparent;">
        Links to the project's sources, along with concise descriptions of how they influenced development.
      </div>
</div>

<div style="display: flex; align-items: stretch; gap: 10px;">
      <div style="flex: 0 0 140px; background-color: #6C3BAA; color:#FFFFFF; padding:10px; border-radius:8px; font-size:16px; text-align: center;">
        <strong>Alternatives</strong>
      </div>
      <div style="flex: 1; border: 3px solid #6C3BAA; padding:12px; border-radius:8px; font-size:13px; color: inherit; background-color: transparent;">
        Compares the solution against other potential approaches, providing a justification for the final choice.
      </div>
</div>

  </div>
</div>


### Imports

In [8]:
# Imports
import hashlib
import os
import struct

### Task 1 - Binary Representations

1.1 Rotate the bits in a 32-bit unsigned integer to the left by `n` places

In [2]:
def rotl(x, n):
    # Get the modulo 32 to avoid unnecessary rotations
    n %= 32

    # Perform the rotation
    return ((x << n) & 0xFFFFFFFF) | (x >> (32 - n))

1.2 Rotate the bits in a 32-bit unsigned integer to the right by `n` places

In [3]:
def rotr(x, n):
    # Get the modulo 32 to avoid unnecessary rotations
    n %= 32

    # Perform the right rotation
    return ((x >> n) & 0xFFFFFFFF) | (x << (32 - n))

1.3 Choose the bits from `y` where `x` has bits set to `1` and bits in `z` where `x` has bits set to `0`

In [4]:
def ch(x, y, z):
    return (y & x) | (z & ~x)

1.4 Take a majority vote of the bits in `x`, `y`, and `z`.

In [5]:
def maj(x, y, z):
    return (x & y) | (z & x) | (z & y)

<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 230px;">
    
##### **Test 1A - "Verifying `rotl`"**  
**We will now test `rotl(x, n)`**

**Expected results:**

| Input  | Output |
|--------|--------|
| `20` `8`   | `5120` |
| `2153` `8` | `551168` |
</div>


In [10]:
print(f"rotl(20, 8) -> {rotl(20, 8)}")
print(f"rotl(2153, 8) -> {rotl(2153, 8)}")

rotl(20, 8) -> 5120
rotl(2153, 8) -> 551168


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 230px;">
    
##### **Test 1B - "Verifying `rotr`"**  
**We will now test `rotr(x, n)`**

**Expected results:**

| Input  | Output |
|--------|--------|
| `20` `8`   | `335544320` |
| `2153` `8` | `36121346056` |
</div>


In [11]:
print(f"rotr(20, 8) -> {rotr(20, 8)}")
print(f"rotr(2153, 8) -> {rotr(2153, 8)}")

rotr(20, 8) -> 335544320
rotr(2153, 8) -> 36121346056


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 230px;">
    
##### **Test 1C - "Verifying `ch`"**  
**We will now test `ch(x, y, z)`**

**Expected results:**

| Input  | Output |
|--------|--------|
| `20` `2153` `54`   | `34` |
</div>


In [12]:
print(f"ch(20, 2153, 54) -> {ch(20, 2153, 54)}")

ch(20, 2153, 54) -> 34


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 230px;">
    
##### **Test 1D - "Verifying `maj`"**  
**We will now test `maj(x, y, z)`**

**Expected results:**

| Input  | Output |
|--------|--------|
| `20` `2153` `54`   | `52` |
</div>


In [13]:
print(f"maj(20, 2153, 54) -> {maj(20, 2153, 54)}")

maj(20, 2153, 54) -> 52


### Task 2: Hash Functions


2.1 Generate a hash value for a string.

In [14]:
def hash_string(s: str) -> int:
    hashval = 0
    for char in s:
        hashval = ord(char) + 31 * hashval
    return hashval % 101

<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 300px;">
    
##### **Test 2A - "Verifying `hash_string`"**  
**We will now test `hash_string(s)`**

**Expected results:**

| Input  | Output |
|--------|--------|
| `Brutus` | `26` |
| `brutus` | `36` |
</div>


In [20]:
print(f"hash_string(Brutus) -> {hash_string('Brutus')}")
print(f"hash_string(brutus) -> {hash_string('brutus')}")

hash_string(Brutus) -> 26
hash_string(brutus) -> 36


### Task 3: SHA256


In [15]:
def calculateSHA256padding(file_path):
    file_size = os.path.getsize(file_path) * 8

    if file_size % 512 < 448:
        padding_bits = 448 - (file_size % 512)
    else:
        padding_bits = 512 - (file_size % 512) + 448

    padding = b'\x80' + b'\x00' * ((padding_bits // 8) - 1)

    original_length = struct.pack('>Q', file_size)
    padding += original_length


    return padding.hex()

calculateSHA256padding("testsha.txt")

'80000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018'

<div style="border: 3px solid #4F83CC; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 390px;">

<b>FIPS PUB 180-4, Secure Hash Standard <a href="https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf" style="color:#1976D2; text-decoration: underline;"></b>(Link)</a>

This document outlines the official standards for SHA (Secure Hash Algorithms), which are widely used to ensure data integrity, secure passwords, and support digital signatures in modern cryptographic systems.
</div>


### Task 4: Prime Numbers

Approach A: Sieve of Eratosthenes

In [2]:
def sieve_of_eratosthenes(limit):
    primes = [True] * (limit + 1)
    primes[0] = primes[1] = False

    for i in range(2, int(limit**0.5) + 1):
        if primes[i]:
            for j in range(i * i, limit + 1, i):
                primes[j] = False

    return [num for num, is_prime in enumerate(primes) if is_prime]


Approach B: Sieve of Sundaram

In [15]:
def sieve_of_sundaram(limit):
    m = (limit - 1) // 2
    sieve = [False] * (m + 1)

    for i in range(1, m + 1):
        j = i
        while i + j + 2 * i * j <= m:
            sieve[i + j + 2 * i * j] = True
            j += 1

    primes = [2] 
    odd_primes = [2 * i + 1 for i in range(1, m + 1) if not sieve[i]]

    primes.extend(odd_primes)

    return [prime for prime in primes if prime <= limit]


<div style="border: 3px solid #5C9E75; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 600px;">
    
##### **Test 3A - "Verifying `sieve_of_sundaram` and `sieve_of_eratosthenes`"**  
We will now test both algorithms and compare their results

Expected results: Both algorithms should have identical outputs
</div>


In [17]:
primes_eratosthenes = sieve_of_eratosthenes(100)

primes_sundaram = sieve_of_sundaram(100)

if primes_eratosthenes == primes_sundaram:
    print("Both methods have identical outputs.")
else:
    print("The methods return different primes.")

Both methods have identical outputs.


<div style="display: flex; gap: 20px;">

  <div style="border: 3px solid #4F83CC; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 330px;">
    <b><a href="https://www.geeksforgeeks.org/sieve-of-eratosthenes/" style="color:#1976D2; text-decoration: underline;">Sieve of Eratosthenes</b></a><br><br>
    This page explains the Sieve of Eratosthenes algorithm for efficiently generating all prime numbers less than a given number, an important method in number theory and cryptography.
  </div>

  <div style="border: 3px solid #4F83CC; padding:8px; border-radius:5px; font-size:14px; background-color: transparent; width: 330px;">
    <a href="https://www.geeksforgeeks.org/sieve-sundaram-print-primes-smaller-n/" style="color:#1976D2; text-decoration: underline;"><b>Sieve of Sundaram</b></a><br><br>
    This page explains the Sieve of Sundaram algorithm for finding prime numbers smaller than a given number, a method often used in number theory and computational applications.
  </div>

</div>

### Task 5: Roots

#### 1.1 First 100 Prime Numbers

Before calculating the square root of the first 100 prime numbers, we need those numbers.

To get the first 100 prime numbers, we can use the `sieve_of_eratosthenes` method from the previous task

In [8]:
first_100_primes = sieve_of_eratosthenes(550)

#### 1.2 Square Root of Each Prime Number

In [9]:
import math

square_roots = [math.sqrt(p) for p in first_100_primes]

#### 1.3 Extract Fractional Part of Square Roots

In [10]:
fractional_parts = [sqrt - int(sqrt) for sqrt in square_roots]

#### 1.4 Calculate First 32 Bits from Each Square Root

In [11]:
def get_first_32_bits(fractional_part):
    return format(int(fractional_part * (2**32)), '032b')  

binary_fractions = [get_first_32_bits(fraction) for fraction in fractional_parts]

#### 1.5 Print Results

In [12]:
for i, binary_fraction in enumerate(binary_fractions):
    print(f"{first_100_primes[i]:>3} -> {binary_fraction}")

  2 -> 01101010000010011110011001100111
  3 -> 10111011011001111010111010000101
  5 -> 00111100011011101111001101110010
  7 -> 10100101010011111111010100111010
 11 -> 01010001000011100101001001111111
 13 -> 10011011000001010110100010001100
 17 -> 00011111100000111101100110101011
 19 -> 01011011111000001100110100011001
 23 -> 11001011101110111001110101011101
 29 -> 01100010100110100010100100101010
 31 -> 10010001010110010000000101011010
 37 -> 00010101001011111110110011011000
 41 -> 01100111001100110010011001100111
 43 -> 10001110101101000100101010000111
 47 -> 11011011000011000010111000001101
 53 -> 01000111101101010100100000011101
 59 -> 10101110010111111001000101010110
 61 -> 11001111011011001000010111010011
 67 -> 00101111011100110100011101111101
 71 -> 01101101000110000010011011001010
 73 -> 10001011010000111101010001010111
 79 -> 11100011011000001011010110010110
 83 -> 00011100010001010110000000000010
 89 -> 01101111000110010110001100110001
 97 -> 11011001010011101011111010110001


### Task 6: Proof of Work

In [6]:
import hashlib

def count_leading_zero_bits(digest):
    count = 0
    for byte in digest:
        for i in range(8):
            if (byte >> (7 - i)) & 1 == 0:
                count += 1
            else:
                return count
    return count

def load_words(filename):
    with open(filename, "r") as f:
        return [line.strip() for line in f if line.strip().isalpha()]

def find_best_words(words):
    max_zeros = 0
    best_words = []

    for word in words:
        digest = hashlib.sha256(word.encode("utf-8")).digest()
        zeros = count_leading_zero_bits(digest)

        if zeros > max_zeros:
            max_zeros = zeros
            best_words = [word]
        elif zeros == max_zeros:
            best_words.append(word)

    return max_zeros, best_words

if __name__ == "__main__":
    word_file = "dictionary.txt"  
    word_list = load_words(word_file)

    max_bits, top_words = find_best_words(word_list)

    print(f"Max leading zero bits: {max_bits}")
    print("Top word(s):")
    for word in top_words:
        print(word)


Max leading zero bits: 18
Top word(s):
goaltenders


### Task 7: Turing Machines

### Task 8: Computational Complexity